In [626]:
# Imports
import pandas as pd
from sqlalchemy import create_engine, text

In [627]:
# Create a sqlalchemy engine to input database
def get_engine(dbname, host='localhost', port=5432, user='nihar'):
    connection_string = f'postgresql://{user}@{host}:{port}/{dbname}'
    engine = create_engine(connection_string)
    return engine

In [628]:
# The function that runs any query; returns results for SELECT queries and commits changes for non-SELECT queries
def execute_query(dbname, query):
    # Establish the connection
    engine = get_engine(dbname)
    
    # Create transaction
    with engine.connect() as connection:
        transaction = connection.begin() 
        try:
            # If SELECT query, fetch and return the results
            if query.strip().lower().startswith('select'):
                result = connection.execute(text(query))
                return result.fetchall()
            
            # Execute non-SELECT queries
            else:
                connection.execute(text(query))
                # Commit the changes and return None
                transaction.commit()
                return None 
        except Exception as e: # Re-raise the exception
            transaction.rollback()  
            raise e 

In [629]:
# Returns user database
def get_users():
    # Query to select all users
    query = "SELECT * FROM users;"
    df = pd.DataFrame(execute_query('users', query))
    return df

In [630]:
def get_id(username):
    # SQL query to get id of user
    query = f"SELECT id FROM users WHERE username = '{username}';"
    id = execute_query('users', query)
    if len(id) == 0: return None  # Username not found in database
    return id[0][0] # Extract data and return

In [631]:
# Functions to add and remove users

def add_user(username, email, password_hash, hostname = 'localhost', port = 5432, postgre_user = 'nihar'):
    # SQL query to insert a new user
    query = f"INSERT INTO users (username, email, password_hash) VALUES ('{username}', '{email}', '{password_hash}');"
    execute_query('users', query)

def remove_user(username, hostname = 'localhost', port = 5432, postgre_user = 'nihar'):
    # SQL query to delete a user based on the username
    id = get_id(username)
    if id == None: return
    query = f"DELETE FROM users WHERE id = '{id}';"
    execute_query('users', query)

In [632]:
# Update observation helper function
def update_info(id, updated_info, category): 
    if id == None: return # If not a valid user, terminate
    
    query = f"UPDATE users SET {category} = '{updated_info}' WHERE id = {id};" # SQL query to update a user's info based on id
    execute_query('users', query)

# Update observation's data functions
def update_username(id, new_username):
    update_info(id, new_username, 'username')

def update_email(id, new_email):
    update_info(id, new_email, 'email')

def update_password(id, new_pass):
    update_info(id, new_pass, 'password_hash')

In [633]:
# Get ID example
print(get_id("jethalal"))

9120275521814384


In [634]:
# Get DB example
get_users()

,id,username,email,password_hash
0,9120275521814384,jethalal,sethji@shaadi.com,password_hash_420


In [635]:
# Add user example
remove_user("chungus") # Ensure doesn't exist since we're asserting users are unique
add_user("chungus", "bingus@yahoo.co.in", "hashed_password_222")
get_users()

,id,username,email,password_hash
0,9120275521814384,jethalal,sethji@shaadi.com,password_hash_420
1,1830010682693057,chungus,bingus@yahoo.co.in,hashed_password_222


In [636]:
# Remove user example
remove_user("chungus")
df = get_users()
df

,id,username,email,password_hash
0,9120275521814384,jethalal,sethji@shaadi.com,password_hash_420


In [637]:
# Update User data example
update_username(get_id("jethalal"), "tapu ke papa")
update_email(get_id("tapu ke papa"), "jijaji@sundar.con")
update_password(get_id("tapu ke papa"), "JalebiMaster")
df = get_users()

# Undo changes
update_username(get_id("tapu ke papa"), "jethalal")
update_email(get_id("jethalal"), "sethji@shaadi.com")
update_password(get_id("jethalal"), "password_hash_420")

df

,id,username,email,password_hash
0,9120275521814384,tapu ke papa,jijaji@sundar.con,JalebiMaster
